<a href="https://colab.research.google.com/github/kevgam/CAS_IE_Information_Retrieval/blob/main/IR_TEST_ipywidgets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation

!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00


In [8]:
# Google Drive verbinden

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Interaktive Suche mit ipywidgets

In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


# **1. Daten laden**
data = pd.read_csv('/content/drive/MyDrive/ie_scripting_datasets/Archive/songs_with_combined_emotions.csv')



# Funktion zur Visualisierung und Anzeige der Songs
def visualize_emotions_and_display_songs(filtered_data, title="Emotion Distribution"):
    # Visualisierung der Emotionen
    plt.figure(figsize=(8, 5))

    # Erstellen der Farbpalette entsprechend der Anzahl der Emotionen
    emotion_count = filtered_data['final_emotion'].nunique()
    palette = sns.color_palette("Set2", emotion_count)  # Beschränke die Palette auf die Anzahl der einzigartigen Emotionen

    # Setze hue auf 'final_emotion' und gebe die Farbpalette an
    sns.countplot(
        data=filtered_data,
        x='final_emotion',
        order=filtered_data['final_emotion'].value_counts().index,
        hue='final_emotion',
        palette=palette,
        legend=False
    )
    plt.title(title)
    plt.xlabel('Emotion')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()

    # Anzeige der Songs
    if not filtered_data.empty:
        display(filtered_data[['name', 'artists', 'final_emotion', 'valence', 'danceability', 'energy', 'loudness', 'tempo', 'lyrics']])
    else:
        print("Keine Songs gefunden. Passen Sie die Filter an.")


# Widgets definieren
emotion_dropdown = widgets.Dropdown(
    options=['All'] + sorted(data['final_emotion'].unique()),
    value='All',
    description='Emotion:',
    style={'description_width': 'initial'}
)

# Bereinigung der 'artists'-Spalte: Ersetze NaN durch 'Unknown' und stelle sicher, dass alle Werte Strings sind
data['artists'] = data['artists'].fillna('Unknown').astype(str)

# Dropdown für Künstler mit alphabetischer Sortierung
artist_dropdown = widgets.Dropdown(
    options=['All'] + sorted(data['artists'].unique()),
    value='All',
    description='Artist:',
    style={'description_width': 'initial'}
)

valence_slider = widgets.FloatRangeSlider(
    value=[0.0, 1.0],
    min=0.0,
    max=1.0,
    step=0.01,
    description='Valence:',
    style={'description_width': 'initial'}
)

danceability_slider = widgets.FloatRangeSlider(
    value=[0.0, 1.0],
    min=0.0,
    max=1.0,
    step=0.01,
    description='Danceability:',
    style={'description_width': 'initial'}
)

# Zusätzliche Spotify-Features
energy_slider = widgets.FloatRangeSlider(
    value=[0.0, 1.0],
    min=0.0,
    max=1.0,
    step=0.01,
    description='Energy:',
    style={'description_width': 'initial'}
)

loudness_slider = widgets.FloatRangeSlider(
    value=[-60, 0],
    min=-60,
    max=0,
    step=0.5,
    description='Loudness:',
    style={'description_width': 'initial'}
)

tempo_slider = widgets.FloatRangeSlider(
    value=[50, 200],
    min=50,
    max=200,
    step=1,
    description='Tempo:',
    style={'description_width': 'initial'}
)

lyrics_search = widgets.Text(
    value='',
    placeholder='Wort eingeben',
    description='Lyrics:',
    style={'description_width': 'initial'}
)

filter_button = widgets.Button(
    description='Filter',
    button_style='success',
    icon='filter'
)

output = widgets.Output()

# Filter-Funktion
def apply_filters(change=None):
    with output:
        clear_output(wait=True)
        filtered_data = data.copy()

        # Emotionen filtern
        if emotion_dropdown.value != 'All':
            filtered_data = filtered_data[filtered_data['final_emotion'] == emotion_dropdown.value]

        # Künstler filtern
        if artist_dropdown.value != 'All':
            filtered_data = filtered_data[filtered_data['artists'] == artist_dropdown.value]

        # Valence filtern
        min_valence, max_valence = valence_slider.value
        filtered_data = filtered_data[
            (filtered_data['valence'] >= min_valence) & (filtered_data['valence'] <= max_valence)
        ]

        # Danceability filtern
        min_danceability, max_danceability = danceability_slider.value
        filtered_data = filtered_data[
            (filtered_data['danceability'] >= min_danceability) & (filtered_data['danceability'] <= max_danceability)
        ]

        # Energy filtern
        min_energy, max_energy = energy_slider.value
        filtered_data = filtered_data[
            (filtered_data['energy'] >= min_energy) & (filtered_data['energy'] <= max_energy)
        ]

        # Loudness filtern
        min_loudness, max_loudness = loudness_slider.value
        filtered_data = filtered_data[
            (filtered_data['loudness'] >= min_loudness) & (filtered_data['loudness'] <= max_loudness)
        ]

        # Tempo filtern
        min_tempo, max_tempo = tempo_slider.value
        filtered_data = filtered_data[
            (filtered_data['tempo'] >= min_tempo) & (filtered_data['tempo'] <= max_tempo)
        ]

        # Lyrics-Suche
        if lyrics_search.value.strip():
            filtered_data = filtered_data[
                filtered_data['clean_lyrics'].str.contains(lyrics_search.value.strip(), case=False, na=False)
            ]

        # Ergebnisse visualisieren und anzeigen
        if not filtered_data.empty:
            print(f"Gefundene Songs: {len(filtered_data)}")
            visualize_emotions_and_display_songs(filtered_data)
        else:
            print("Keine Songs gefunden. Passen Sie die Filter an.")

filter_button.on_click(apply_filters)

# Layout der Widgets
ui = widgets.VBox([
    widgets.Label("Filtere Songs basierend auf Emotionen, Künstler, Spotify-Features und Lyrics:"),
    emotion_dropdown,
    artist_dropdown,
    valence_slider,
    danceability_slider,
    energy_slider,
    loudness_slider,
    tempo_slider,
    lyrics_search,
    filter_button,
    output
])

# Zeige das Widget an
display(ui)